In [3]:
from pathlib import Path

from helper import cut_pages, merge_pdfs

from docling.document_converter import DocumentConverter  # 

from llama_index.readers.docling import DoclingReader      # 
from llama_index.node_parser.docling import DoclingNodeParser  # 
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "data"
DATA_DIR.mkdir(exist_ok=True)

big_book = DATA_DIR / "green-line-1-g9.pdf"  # put your 120-page PDF here


In [ ]:
snippets = []

# Example ranges – adjust for your book
snippets.append(
    cut_pages(big_book, pages=list(range(13,35)), output_pdf=DATA_DIR / "toy_intro_1-3.pdf")
)
snippets.append(
    cut_pages(big_book, pages=[10, 11, 12, 13], output_pdf=DATA_DIR / "toy_unit1_10-13.pdf")
)
snippets.append(
    cut_pages(big_book, pages=[40, 41], output_pdf=DATA_DIR / "toy_exercises_40-41.pdf")
)

toy_pdf = merge_pdfs(snippets, DATA_DIR / "toy_green_line_1.pdf")
toy_pdf
